In [3]:
import pandas as pd
import lightfm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, reciprocal_rank, auc_score
from lightfm.cross_validation import random_train_test_split
from dataloading import DataLoader
from datapreparation_py import DataPreparation
from lightFM_adoptor import LightFMAdaptor
from cv_py import CrossValidation
from implicit.evaluation import ranking_metrics_at_k, train_test_split

c:\Users\781110104\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [1]:
%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [4]:
dl = DataLoader()
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [5]:
user_item_co_t_filtered = dl.remove_low_interact_items(user_item_co_t, 1)

In [6]:
dp = DataPreparation(user_item_co_t_filtered)

In [11]:
interactions_csr, weights_csr = dp.get_interaction_data()

In [12]:
user_features = ['country']
item_features = ['group2']
feature_input = dp.get_feature_input(user_features=user_features, item_features=item_features)

In [10]:
userf, itemf = feature_input

In [13]:
train_i, test_i = train_test_split(interactions_csr, 0.8, 22)
train_w, test_w = train_test_split(weights_csr, 0.8, 22)

In [25]:
cv_i = CrossValidation(train_i, 5)
cv_w = CrossValidation(train_w, 5)

In [26]:
val_i, cal_i = cv_i.split_k_fold(22)
val_w, cal_w = cv_w.split_k_fold(22)

In [27]:
model_test = LightFM(loss='warp', random_state=22, no_components=5)
model_test.fit(interactions=cal_i['0'], user_features=userf, item_features=itemf, sample_weight=cal_w['0'].tocoo())

In [28]:
test_pk = precision_at_k(model_test, test_interactions=val_i['0'], user_features=userf, item_features=itemf, train_interactions=cal_i['0'], k=10).mean()
test_pk

0.097500004

In [29]:
model_adapt = LightFMAdaptor(loss='warp', random_state=22, no_components=5)

In [30]:
model_adapt.fit(interactions=cal_i['0'], user_features=userf, item_features=itemf, weights=cal_w['0'].tocoo(), iterations=1)

Epoch: 100%|██████████| 1/1 [00:00<00:00,  5.33it/s]


In [31]:
ranking_metrics_at_k(model_adapt, cal_i['0'], val_i['0'], 10)

  0%|          | 0/40 [00:00<?, ?it/s]

{'precision': 0.0975,
 'map': 0.03615277777777778,
 'ndcg': 0.09260596865784895,
 'auc': 0.5019524090042894}